# Importing the Libraries

In [3]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# Data Pre-processing

### 1. Get Notes Function

In [0]:
def get_notes():
    # Get all the notes and chords from the midi files in the ./midi_songs directory
    notes = []

    for file in glob.glob(path + "midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(path + 'data/notes.pickle', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


### 2. Prepare Sequences Function

In [0]:
def prepare_sequences(notes, n_vocab):
    # Prepare the sequences used by the Neural Network
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


# Creating the Model

### 1. Model Definition

In [0]:
def create_network(network_input, n_vocab):
    # Create the structure of the Neural Network
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


### 2. Compile & Train the Model

In [0]:
def train(model, network_input, network_output):
    # Train the Neural Network.
    filepath = path + "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=1024, callbacks=callbacks_list)

# Driver Code to Process the Data

In [8]:
# Set GDrive Path
path = '/content/drive/My Drive/Datasets/lstm-music/'

# Train a Neural Network to generate music
notes = get_notes()

# get amount of pitch names
n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)

Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/electric_de_chocobo.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/dontbeafraid.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/In_Zanarkand.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/costadsol.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/Oppressed.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/caitsith.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/FFIXQuMarshP.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/great_war.mid
Parsing /content/drive/My Drive/Datasets/lstm-music/midi_songs/braska.mid


# Driver code to Create & Train Model 

In [11]:
model = create_network(network_input, n_vocab)

train(model, network_input, network_output)

Epoch 1/200
57077/57077 [==============================] - 47s 819us/step - loss: 5.5942
Epoch 2/200
57077/57077 [==============================] - 46s 803us/step - loss: 5.1849
Epoch 3/200
57077/57077 [==============================] - 46s 799us/step - loss: 5.1276
Epoch 4/200
57077/57077 [==============================] - 45s 796us/step - loss: 4.7624
Epoch 5/200
57077/57077 [==============================] - 46s 799us/step - loss: 4.7339
Epoch 6/200
57077/57077 [==============================] - 46s 798us/step - loss: 4.6943
Epoch 7/200
57077/57077 [==============================] - 45s 794us/step - loss: 4.5976
Epoch 8/200
57077/57077 [==============================] - 46s 799us/step - loss: 4.8729
Epoch 9/200
57077/57077 [==============================] - 45s 796us/step - loss: 4.5963
Epoch 10/200
57077/57077 [==============================] - 45s 797us/step - loss: 4.7357
Epoch 11/200
57077/57077 [==============================] - 45s 797us/step - loss: 4.6058
Epoch 12/200
57077/